# Pretrained One-vs-Rest Classifier

This notebook implements an one-vs-rest classifier that fine-tunes several BERT models to tell if a sentence contains problematic metaphors. It starts out from our BERT model pretrained on a large corpus of data.

<div hidden>
TODO: add extend data3/data.json with better data in the same format that actually makes sense.
</div>

## Imports and Setup

In [1]:
%pip install transformers -Uqq
%pip install sklearn -Uqq
%pip install datasets -Uqq
%pip install torch -Uqq
%pip install numpy -Uqq
%pip install evaluate -Uqq

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import evaluate
import numpy as np
import torch
from datasets import Dataset, load_dataset
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    EvalPrediction,
    Trainer,
    TrainingArguments,
)
import os
from itertools import product

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


## Loading Dataset

In [3]:
dataset = load_dataset("json", data_files="data/data.json", field="data")
dataset

Found cached dataset json (/home/xt0r3/.cache/huggingface/datasets/json/default-08d0520c01c5ad03/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'agency', 'humanComparison', 'hyperbole', 'historyComparison', 'unjustClaims', 'deepSounding', 'sceptics', 'deEmphasize', 'performanceNumber', 'inscrutable'],
        num_rows: 1022
    })
})

In [4]:
dataset["train"][0:3]

{'text': ['A new vision of artificial intelligence for the people',
  'The gig workers fighting back against the algorithms',
  'How the AI industry profits from catastrophe'],
 'agency': [False, True, False],
 'humanComparison': [False, True, False],
 'hyperbole': [False, True, True],
 'historyComparison': [False, False, False],
 'unjustClaims': [False, False, False],
 'deepSounding': [False, False, False],
 'sceptics': [False, False, False],
 'deEmphasize': [False, False, False],
 'performanceNumber': [False, False, False],
 'inscrutable': [False, False, False]}

In [5]:
labels = [label for label in dataset["train"].features.keys() if label not in ["text"]]

num_epochs = {
    "agency": 10,
    "humanComparison": 2,
    "hyperbole": 2,
    "uncriticalHistoryComparison": 2,
    "unjustifiedClaimsAboutFuture": 5,
    "deepSounding": 2,
    "sceptics": 2,
    "deEmphasize": 7,
    "performanceNumber": 2,
    "inscrutable": 2,
}

labels

['agency',
 'humanComparison',
 'hyperbole',
 'historyComparison',
 'unjustClaims',
 'deepSounding',
 'sceptics',
 'deEmphasize',
 'performanceNumber',
 'inscrutable']

## Preprocess Data, Create Train/Test Split

In [6]:
processed_dataset = {}
for label in labels:
    projected_dataset = (
        dataset["train"]
        .map(remove_columns=[l for l in labels if l != label])
        .rename_column(label, "labels")
        .class_encode_column("labels")
    )
    processed_dataset[label] = projected_dataset.train_test_split(
        test_size=0.2, stratify_by_column="labels"
    )
    # print(f"{label}:\n\t{processed_dataset[label]['test'][0:3]}\n")

processed_dataset

Loading cached processed dataset at /home/xt0r3/.cache/huggingface/datasets/json/default-08d0520c01c5ad03/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-faae32b27e25bfec.arrow
Loading cached processed dataset at /home/xt0r3/.cache/huggingface/datasets/json/default-08d0520c01c5ad03/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-aa385659ca8667d4.arrow
Loading cached processed dataset at /home/xt0r3/.cache/huggingface/datasets/json/default-08d0520c01c5ad03/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-848a2cf3fb2b0f4d.arrow
Loading cached processed dataset at /home/xt0r3/.cache/huggingface/datasets/json/default-08d0520c01c5ad03/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-9d22fa26c632e3c9.arrow
Loading cached processed dataset at /home/xt0r3/.cache/huggingface/datasets/json/default-08d0520c01c5ad03/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51

{'agency': DatasetDict({
     train: Dataset({
         features: ['text', 'labels'],
         num_rows: 817
     })
     test: Dataset({
         features: ['text', 'labels'],
         num_rows: 205
     })
 }),
 'humanComparison': DatasetDict({
     train: Dataset({
         features: ['text', 'labels'],
         num_rows: 817
     })
     test: Dataset({
         features: ['text', 'labels'],
         num_rows: 205
     })
 }),
 'hyperbole': DatasetDict({
     train: Dataset({
         features: ['text', 'labels'],
         num_rows: 817
     })
     test: Dataset({
         features: ['text', 'labels'],
         num_rows: 205
     })
 }),
 'historyComparison': DatasetDict({
     train: Dataset({
         features: ['text', 'labels'],
         num_rows: 817
     })
     test: Dataset({
         features: ['text', 'labels'],
         num_rows: 205
     })
 }),
 'unjustClaims': DatasetDict({
     train: Dataset({
         features: ['text', 'labels'],
         num_rows: 817
     })
  

In [7]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def preprocess_data(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [8]:
tokenized_dataset = {
    k: ds.map(
        preprocess_data,
        remove_columns="text",
        batched=True,
    )
    for k, ds in processed_dataset.items()
}

Map:   0%|          | 0/817 [00:00<?, ? examples/s]

Map:   0%|          | 0/205 [00:00<?, ? examples/s]

Map:   0%|          | 0/817 [00:00<?, ? examples/s]

Map:   0%|          | 0/205 [00:00<?, ? examples/s]

Map:   0%|          | 0/817 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/205 [00:00<?, ? examples/s]

Map:   0%|          | 0/817 [00:00<?, ? examples/s]

Map:   0%|          | 0/205 [00:00<?, ? examples/s]

Map:   0%|          | 0/817 [00:00<?, ? examples/s]

Map:   0%|          | 0/205 [00:00<?, ? examples/s]

### Verify dataset

In [9]:
example = tokenized_dataset["agency"]["train"][0]
print(example.keys())

dict_keys(['labels', 'input_ids', 'token_type_ids', 'attention_mask'])


In [10]:
tokenizer.decode(example["input_ids"])

'[CLS] NASA invites you to explore Mars though images of the Red Planet taken by its Curiosity rover and help train its AI algorithm by labelling rocks and surface features as you go [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

In [11]:
example["labels"]

0

## Load Pre-Trained Model

In [12]:
# use_fast uses fast tokenizers backed by rust. Remove it if it causes errors
# model = AutoModelForSequenceClassification.from_pretrained(
    # "bert-base-cased",
    # num_labels=2,
# )

### Verify data-model interaction

In [13]:
# forward pass
# outputs = model(
# input_ids=tokenized_dataset[labels[0]]["train"]["input_ids"][0],
# labels=tokenized_dataset[labels[0]]["train"][0]["labels"],
# )
# outputs

## Define Metrics

In [14]:
metrics = {
    "accuracy": evaluate.load("accuracy"),
    "presicion": evaluate.load("precision"),
    "recall": evaluate.load("recall"),
    "f1": evaluate.load("f1"),
}

In [15]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    values = {}
    
    for name, metric in metrics.items():
        result = metric.compute(predictions=predictions, references=labels)
        for val in result.values() if isinstance(result, dict) else [result]:
            values[name] = val

    return values

In [16]:
class HighPrecisionTrainer(Trainer):
    """A trainer class, which computes loss based on a weighted MSE, where the error for the positive labels is
    weighted more than the error for the negative labels, leading to a higher precision
    """

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        label_mask = torch.FloatTensor(1, 2).cuda().zero_()
        label_mask[0, labels[0]] = 1
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # compute custom loss (suppose one has 3 labels with different weights)
        loss = torch.sum(torch.tensor([[1, 300]]).cuda() * ((logits - label_mask) ** 2))
        return (loss, outputs) if return_outputs else loss

## Train the Model

In [17]:
  # TODO: increase if we have more data
metric_name = "f1"

In [18]:
for model_name, weight_decay in product(['xt0r3/aihype_article_bert_fine_tune', 'bert-base-cased'], [0.0]):  # , 0.01]):
    for i in range(0, 4):
        batch_size = 2 ** i
        print(f'batch size: {2 ** i}\nmodel: {model_name}\nweight decay: {weight_decay}')

        for label in ['agency']:  # labels:
            print(f"training model for {label}")

            model = AutoModelForSequenceClassification.from_pretrained(
                model_name,
                num_labels=2,
            )

            training_args = TrainingArguments(
                f"{model_name.replace('/', '_')}_{label}_{i}-vs-rest",
                evaluation_strategy="epoch",
                save_strategy="epoch",
                learning_rate=2e-5,
                per_device_train_batch_size=batch_size,
                per_device_eval_batch_size=batch_size,
                num_train_epochs=num_epochs[label],
                weight_decay=weight_decay,
                report_to="none",
                load_best_model_at_end=True,
                metric_for_best_model=metric_name,
                save_total_limit = 1,
                # push_to_hub=True,  # TODO: enable once model seems good
            )

            trainer = Trainer(
                model=model,
                args=training_args,
                train_dataset=tokenized_dataset[label]["train"],
                eval_dataset=tokenized_dataset[label]["test"],
                compute_metrics=compute_metrics,
            )

            trainer.train()

batch size: 1
model: xt0r3/aihype_article_bert_fine_tune
weight decay: 0.0
training model for agency


Some weights of the model checkpoint at xt0r3/aihype_article_bert_fine_tune were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at xt0r3/aihype_article_bert_fine_tune and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inferen

Epoch,Training Loss,Validation Loss,Accuracy,Presicion,Recall,F1
1,1.112100,1.012461,0.751220,0.500000,0.627451,0.556522
2,0.753100,0.987382,0.795122,0.615385,0.470588,0.533333
3,0.374900,1.356253,0.809756,0.636364,0.549020,0.589474
4,0.047400,1.563559,0.824390,0.636364,0.686275,0.660377
5,0.042100,1.959850,0.775610,0.533333,0.784314,0.634921
6,0.009100,1.613110,0.814634,0.632653,0.607843,0.620000
7,0.007200,1.710962,0.824390,0.653061,0.627451,0.640000
8,0.014800,1.772866,0.824390,0.653061,0.627451,0.640000
9,0.000000,1.784649,0.824390,0.653061,0.627451,0.640000
10,0.012300,1.792016,0.824390,0.653061,0.627451,0.640000


***** Running Evaluation *****
  Num examples = 205
  Batch size = 1
Saving model checkpoint to xt0r3_aihype_article_bert_fine_tune_agency_0-vs-rest/checkpoint-817
Configuration saved in xt0r3_aihype_article_bert_fine_tune_agency_0-vs-rest/checkpoint-817/config.json
Model weights saved in xt0r3_aihype_article_bert_fine_tune_agency_0-vs-rest/checkpoint-817/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 205
  Batch size = 1
Saving model checkpoint to xt0r3_aihype_article_bert_fine_tune_agency_0-vs-rest/checkpoint-1634
Configuration saved in xt0r3_aihype_article_bert_fine_tune_agency_0-vs-rest/checkpoint-1634/config.json
Model weights saved in xt0r3_aihype_article_bert_fine_tune_agency_0-vs-rest/checkpoint-1634/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 205
  Batch size = 1
Saving model checkpoint to xt0r3_aihype_article_bert_fine_tune_agency_0-vs-rest/checkpoint-2451
Configuration saved in xt0r3_aihype_article_bert_fine_tune_agency_0-vs-rest/chec

batch size: 2
model: xt0r3/aihype_article_bert_fine_tune
weight decay: 0.0
training model for agency


...

***** Running Evaluation *****
  Num examples = 205
  Batch size = 1
Saving model checkpoint to bert-base-cased_agency_0-vs-rest/checkpoint-817
Configuration saved in bert-base-cased_agency_0-vs-rest/checkpoint-817/config.json
Model weights saved in bert-base-cased_agency_0-vs-rest/checkpoint-817/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 205
  Batch size = 1
Saving model checkpoint to bert-base-cased_agency_0-vs-rest/checkpoint-1634
Configuration saved in bert-base-cased_agency_0-vs-rest/checkpoint-1634/config.json
Model weights saved in bert-base-cased_agency_0-vs-rest/checkpoint-1634/pytorch_model.bin
Deleting older checkpoint [bert-base-cased_agency_0-vs-rest/checkpoint-817] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 205
  Batch size = 1
Saving model checkpoint to bert-base-cased_agency_0-vs-rest/checkpoint-2451
Configuration saved in bert-base-cased_agency_0-vs-rest/checkpoint-2451/config.json
Model weights saved in bert-bas

batch size: 2
model: bert-base-cased
weight decay: 0.0
training model for agency


loading configuration file config.json from cache at /home/xt0r3/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading weights file pytorch_model.bin from cache at /home/xt0r3/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/pyt

Epoch,Training Loss,Validation Loss,Accuracy,Presicion,Recall,F1
1,No log,0.699402,0.770732,1.000000,0.078431,0.145455
2,0.741600,0.776101,0.819512,0.718750,0.450980,0.554217
3,0.592700,0.886389,0.800000,0.583333,0.686275,0.630631
4,0.357900,1.050515,0.819512,0.694444,0.490196,0.574713
5,0.128300,1.444456,0.790244,0.583333,0.549020,0.565657
6,0.128300,1.757508,0.756098,0.507246,0.686275,0.583333
7,0.045900,1.547263,0.785366,0.581395,0.490196,0.531915
8,0.023200,1.594403,0.795122,0.604651,0.509804,0.553191


***** Running Evaluation *****
  Num examples = 205
  Batch size = 2
Saving model checkpoint to bert-base-cased_agency_1-vs-rest/checkpoint-409
Configuration saved in bert-base-cased_agency_1-vs-rest/checkpoint-409/config.json
Model weights saved in bert-base-cased_agency_1-vs-rest/checkpoint-409/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 205
  Batch size = 2
Saving model checkpoint to bert-base-cased_agency_1-vs-rest/checkpoint-818
Configuration saved in bert-base-cased_agency_1-vs-rest/checkpoint-818/config.json
Model weights saved in bert-base-cased_agency_1-vs-rest/checkpoint-818/pytorch_model.bin
Deleting older checkpoint [bert-base-cased_agency_1-vs-rest/checkpoint-409] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 205
  Batch size = 2
Saving model checkpoint to bert-base-cased_agency_1-vs-rest/checkpoint-1227
Configuration saved in bert-base-cased_agency_1-vs-rest/checkpoint-1227/config.json
Model weights saved in bert-base-c

## Upload the Model

In [ ]:
# agency-vs-rest/checkpoint-263: 0.75 precision, 0.85 recall
#

In [ ]:
# trainer.push_to_hub()